# Reproduce the analysis in the paper

This notebook describes the steps necessary to analyze the experimental results and get the plots in the paper.
_To cut run time, the pre-computed results are in a dedicated folder: `src/utils.RESULTS_DIR`. The corresponding cells are commented out._

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns

from functools import reduce
from pathlib import Path
from tqdm.notebook import tqdm

import src.utils as u
import src.rank_metrics as rm
import src.rank_utils as ru

if Path(os.getcwd()).name != "EncoderBenchmarking":
    os.chdir("..")  # move kernel to EncoderBenchmarking

# Results loading

Load the results and the precomputed rankings, stored in the `src/utils.RESULTS_DIR` directory (default: `experimental_results`).

In [ ]:
df = pd.read_parquet(u.RESULTS_DIR / "results.parquet")
df.info()

Compute the rankings

In [ ]:
rf = ru.get_rankings(df, factors=["dataset", "model", "tuning", "scoring"], alternatives="encoder", target="cv_score",
                     increasing=True, impute_missing=False)
rf.to_parquet(u.RESULTS_DIR / "rankings.parquet")
rf.info()

## Sensitivity analysis
The relative performance of encoders can depend on an ML model, a quality metric, or a tuning strategy. The choice of an aggregation strategy impacts the consensus ranking.
To quantify the influence of these choices, we calculate the similarity between rankings using the Jaccard index $J$ for the sets of best encoders as well as the Spearman correlation coefficient $\rho$.
For both measures, values close to 1 indicate high agreement between rankings and low sensitivity. Conversely, values near 0 (or, for $\rho$, negative) suggest low consistency and high sensitivity.

### Sensitivity to experimental factors
We evaluate the sensitivity of encoder rankings on individual datasets with respect to an experimental factor (ML model, quality metric, or tuning strategy) by varying the factor of interest and keeping the other factors fixed, then calculating the similarity between pairs of rankings. After that, we average the result across all combinations of the other factors.
Our findings highlight the high sensitivity of results of studies comparing encoders to experimental factors, for both the full rankings and the best encoders.
They also explain why results from other studies are not comparable, as choosing different values for any factor will lead to different results.

In [ ]:
"""
%%time
jaccard_model, rho_model = u.pairwise_similarity_wide_format(rf, simfuncs=[rm.jaccard_best, rm.spearman_rho], shared_levels=[0, 2, 3])
jaccard_tuning, rho_tuning = u.pairwise_similarity_wide_format(rf, simfuncs=[rm.jaccard_best, rm.spearman_rho], shared_levels=[0, 1, 3])
jaccard_scoring, rho_scoring = u.pairwise_similarity_wide_format(rf, simfuncs=[rm.jaccard_best, rm.spearman_rho], shared_levels=[0, 1, 2])

jaccard = reduce(lambda x, y: x.fillna(y), [jaccard_model, jaccard_tuning, jaccard_scoring])
rho = reduce(lambda x, y: x.fillna(y), [rho_model, rho_tuning, rho_scoring])

jaccard.to_parquet(u.ANALYSIS_DIR / "pw_jaccard.parquet")
rho.to_parquet(u.ANALYSIS_DIR / "pw_rho.parquet")
"""

Plot the sensitivity of results to experimental factors.

In [ ]:
sims = u.load_similarity_dataframes()
rho = sims["pw_rho"]
jaccard = sims["pw_jaccard"]

factors = ["model", "tuning", "scoring"]
similarities = ["rho", "jaccard"]

sns.set(font_scale=0.8)
sns.set_style("ticks")
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble'] = r'\usepackage{mathptmx}'
mpl.rc('font', family='Times New Roman')

fig, axes = plt.subplots(1, len(factors), figsize=(5, 3))
for ax, factor in zip(axes, factors):
    df_sim = u.join_wide2long({"jaccard": jaccard, "rho": rho}, comparison_level=factor)
    title = factor
    u.heatmap_longformat_multisim(df_sim, similarities, factor, fontsize=8, annot_fontsize=8, ax=ax)

sns.despine()
plt.tight_layout(pad=0.5)
fig.savefig(u.FIGURES_DIR / "heatmap_model_tuning_scoring.pdf")
plt.show()

### Sensitivity to aggregation strategy

To evaluate the impact of the aggregation strategy on the consensus ranking, we apply the same procedure as above to consensus rankings instead of rankings on individual datasets.
For example, Spearman's $\rho$ between consensus rankings obtained with Q-M and Q-Md averaged across all ML models, tuning strategies, and quality metrics is 0.8.

While some aggregation strategies show strong similarities, different strategies yield very different consensus rankings in general.
This is particularly evident for Jaccard index $J$, indicating the high sensitivity of the best encoders to the rank aggregation strategy.

We begin by computing the consensus rankings with different aggregation strategies.

In [ ]:
# %%time
# consensuses = list()
#
# # --- Aggregation for all aggregation strategies except Nemenyi
# consensuses.append(ru.Aggregator(df, rf).aggregate(verbose=False, ignore_strategies=["nemenyi rank"]).aggrf)
#
# # --- Aggregation with Nemenyi with different significance levels
# for alpha in tqdm([0.01, 0.05, 0.1]):
#     agg = ru.Aggregator(df, rf)
#     bag = agg.base_aggregators[('SVC', 'full', 'AUC')]
#     consensuses.append(agg.aggregate(verbose=False, strategies=["nemenyi rank"], alpha=alpha).aggrf)
#
# aggrf = pd.concat(consensuses, axis=1)
# aggrf.to_parquet(u.ANALYSIS_DIR / "consensuses.parquet")

We then compute the similarity between different consensuses ...

In [ ]:
aggrf = u.load_aggrf().rename(columns=u.AGGREGATION_NAMES, level="aggregation")
agg_jaccard, agg_rho = u.pairwise_similarity_wide_format(aggrf, simfuncs=[rm.jaccard_best, rm.spearman_rho], shared_levels=slice(-1))

agg_jaccard.to_parquet(u.ANALYSIS_DIR / "pw_AGG_jaccard.parquet")
agg_rho.to_parquet(u.ANALYSIS_DIR / "pw_AGG_rho.parquet")

... and finally plot it.

In [ ]:
aggsims = u.load_aggregated_similarity_dataframes()
agg_jaccard = aggsims["pw_AGG_jaccard"]
agg_rho = aggsims["pw_AGG_rho"]

df_sim = u.join_wide2long({"rho": agg_rho, "jaccard": agg_jaccard}, comparison_level="aggregation")

sns.set_style("ticks")
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble'] = r'\usepackage{mathptmx}'
mpl.rc('font', family='Times New Roman')

fig, ax = plt.subplots(1, 1, figsize=(3, 3.3))
u.heatmap_longformat_multisim(df_sim, ["rho", "jaccard"], "aggregation", fontsize=7, annot_fontsize=7, ax=ax)
plt.tight_layout()

fig.savefig(u.FIGURES_DIR / f"heatmap_aggregation.pdf")
plt.show()

## Replicability

Replicability is the property of a benchmark of obtaining consistent results from different data. We quantify replicability by comparing consensus rankings aggregated on two disjoint samples of datasets of equal size.
Each data point represents 100 iterations of randomly selecting two disjoint sets of datasets and applying multiple aggregation strategies. We exclude R-Kem due to its high computational complexity.
Even with 25 datasets, replicability is moderate: this suggests that consensus rankings vary significantly depending on the data.
Results from logistic regression tend to exhibit the highest replicability among ML models, while decision trees show the lowest.

The next cells perform the replicability analysis and plot the results.

In [ ]:
"""
%%time
df = pd.read_parquet(u.RESULTS_DIR / "results.parquet")
rf = pd.read_parquet(u.RESULTS_DIR / "rankings.parquet")

for tuning in ["no", "model", "full"]:
    a = ru.replicability_analysis(df, rf, tuning, sample_sizes=[5, 10, 15, 20, 25], repetitions=100, seed=1444, append_to_existing=False, save=True, ignore_strategies=["nemenyi rank", "kemeny rank"])
"""

In [ ]:
sample_df_sim = pd.concat([pd.read_parquet(u.ANALYSIS_DIR / f"sample_sim_{tuning}.parquet") for tuning in ("no", "model", "full")], axis=0)
sample_df_sim

We now plot the replicability conditional on either model, scoring, or aggregation.

In [ ]:
from importlib import reload
reload(u)


factor = "scoring"  # choices: model, scoring, aggregation
fig = u.lineplot_replicability(sample_df_sim, hue=factor, show=True)

fig.savefig(u.FIGURES_DIR / f"lineplot_replicability_{factor}.pdf")
plt.show()

## Comparing encoders
We now examine the ranks of encoders limited to decision trees, logistic regression, and all ML models.

The left subplot shows the rank of encoders from the experiments with decision trees across all datasets, quality metrics, and tuning strategies.
One-Hot is the best-performing encoder; however, Nemenyi tests at a significance level of 0.05 fail to reject that the average rank of One-Hot is the same as that of the other encoders.

The central subplot features the encoder ranks for logistic regression, where four encoders, namely One-Hot, Sum, Binary, and Weight of Evidence, consistently achieve higher ranks compared to the others.
Nemenyi tests confirm that this difference in ranks is significant.
These results indicate low replicability of the results for decision trees and higher replicability for logistic regression.

The right plot presents the ranks of encoders across all datasets, ML models, quality metrics, and tuning strategies.
Similarly to logistic regression, One-Hot, Sum, Binary, and Weight of Evidence consistently achieve significantly higher average ranks compared to the other encoders, again confirmed by Nemenyi tests.
We recommend these four encoders as the preferred choices in practical applications.
This conclusion contradicts other studies reporting a suboptimal performance of One-Hot~\cite{cerda_similarity_2018, pargent_regularized_2022}.

Our findings also reveal that Drop performs significantly worse than all other encoders, i.e., encoding categorical attributes generally yields better results than dropping them.

In [ ]:
rf = pd.read_parquet(u.RESULTS_DIR / "rankings.parquet")
models = [None, "DTC", "LR", "KNC", "LGBMC", "SVC"]

sns.set(font_scale=0.8)
sns.set_style("ticks", {"ytick.left": False})
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble'] = r'\usepackage{mathptmx}'
mpl.rc('font', family='Times New Roman')

for model in models:
    fig, ax = plt.subplots(1, 1, figsize=(1.8, 4.4))
    u.boxplot_encoder_ranks(rf, ax, model)
    if model is None:
        ax.set_title("all models")
    else:
        ax.set_title(u.FACTOR_LATEX["model"][model])
    ax.set_xlabel("rank")
    sns.despine(left=True, trim=True)
    plt.tight_layout(w_pad=0.5)

    fig.savefig(u.FIGURES_DIR / f"boxplot_rank_{model}.pdf")

The next cell produces the plot for the GitHub repo.

In [ ]:
rf = pd.read_parquet(u.RESULTS_DIR / "rankings.parquet")
models = [None, "DTC", "LR"]

sns.set(font_scale=0.8)
sns.set_style("ticks", {"ytick.left": False})
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble'] = r'\usepackage{mathptmx}'
mpl.rc('font', family='Times New Roman')

fig, axes = plt.subplots(1, len(models))
for ax, model in zip(axes.flatten(), models):
    u.boxplot_encoder_ranks(rf, ax, model)
    if model is None:
        ax.set_title("all models")
    else:
        ax.set_title(u.FACTOR_LATEX["model"][model])
    ax.set_xlabel("rank")
sns.despine(left=True, trim=True)
plt.tight_layout(w_pad=0.5)

fig.savefig(u.FIGURES_DIR / "encoder_ranks.png")
plt.show()

## Missing evaluations

The following cell computes the number of missing evaluations.

In [ ]:
import src.config as cfg

from itertools import product


total_runs = {}
for tuning_strategy in ["no tuning", "model tuning", "full tuning"]:
    total_runs[tuning_strategy] = product(cfg.ENCODERS,
                                          cfg.DATASET_IDS[tuning_strategy],
                                          cfg.MODELS[tuning_strategy],
                                          cfg.SCORINGS)
num_runs = sum(len(list(x)) for x in total_runs.values())

print(f"Total runs     : {num_runs}")
print(f"Completed runs : {len(df)}")

## Runtime

The following cells studies the runtime of encoders.

In [ ]:
df = pd.read_parquet(u.RESULTS_DIR / "results.parquet")
df_ = df.copy()
df_["encoder"] = df_["encoder"].map(u.ENCODER_LATEX)

fig, ax = plt.subplots(1, 1, figsize=(5, 2))
ax = u.boxplot_runtime(df_.query("tuning == 'full'"), ax=ax)
sns.despine(bottom=True, trim=True)
plt.tight_layout(w_pad=0.5)
fig.savefig(u.FIGURES_DIR / "boxplot_tuning_time.pdf")

fig, ax = plt.subplots(1, 1, figsize=(5, 2))
ax = u.boxplot_runtime(df_.query("tuning != 'full'"), ax=ax)
sns.despine(bottom=True, trim=True)
plt.tight_layout(w_pad=0.5)
fig.savefig(u.FIGURES_DIR / "boxplot_encoding_time.pdf")

plt.show()

## Tuning

This cell investigates if tuning lead to an increase in pipeline performance wrt. no tuning.

In [ ]:
df = pd.read_parquet(u.RESULTS_DIR / "results.parquet")
df_ = u.get_dataset_tuning_comparison(df)

sns.set(font_scale=0.8)
sns.set_style("ticks")
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble'] = r'\usepackage{mathptmx}'
mpl.rc('font', family='Times New Roman')

factor = "scoring"
df_[factor] = df_[factor].map(u.FACTOR_LATEX[factor])
fig, axes = plt.subplots(1, 2, figsize=(2.5, 2), sharey="all")
for it, ((tuning_x, tuning_y), ax) in enumerate(zip([("full", "no"), ("full", "model")], axes)):
    ax = sns.boxplot(data=df_.query("tuning_x == @tuning_x and tuning_y == @tuning_y"), x=factor, y="gain",
                                 # palette=sns.light_palette("grey", n_colors=len(rf.index)),
                                 color="lightgrey",
                                 showfliers=False,
                                 linewidth=1, showcaps=False,
                                 showmeans=True,
                                 meanprops={"marker": "o",
                                            "markeredgecolor": "red",
                                            "markersize": 2},
                                 medianprops={"linestyle": "-"
                                              },
                                 ax=ax)
    ax.grid(axis="y", zorder=-1, linewidth=0.4)

    ax.set_ylabel("")
    ax.set_xlabel("")
    ax.set_xticklabels(
        ax.get_xticklabels(),
        rotation=90,
    )

    if it != 0:
        ax.tick_params(left=False)

    ax.set_title(f"{tuning_x} VS {tuning_y}")

sns.despine()
plt.tight_layout(pad=0.5)

plt.savefig(u.FIGURES_DIR / f"boxplot_tuningeffect_{factor}.pdf")

In [ ]:
df = pd.read_parquet(u.RESULTS_DIR / "results.parquet")
df_ = u.get_dataset_tuning_comparison(df)

sns.set(font_scale=0.8)
sns.set_style("ticks")
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble'] = r'\usepackage{mathptmx}'
mpl.rc('font', family='Times New Roman')

df_["encoder"] = df_["encoder"].map(u.ENCODER_LATEX)
for tuning_x, tuning_y in [("full", "no"), ("full", "model")]:
    fig, ax = plt.subplots(1, 1, figsize=(5, 2))
    ax = u.sorted_boxplot_vertical(data=df_.query("tuning_x == @tuning_x and tuning_y == @tuning_y"), x="encoder", y="gain", order_by="mean",
                                 # palette=sns.light_palette("grey", n_colors=len(rf.index)),
                                 color="lightgrey",
                                 showfliers=False,
                                 linewidth=1, showcaps=False,
                                 showmeans=True,
                                 meanprops={"marker": "o",
                                            "markeredgecolor": "red",
                                            "markersize": 2},
                                 medianprops={"linestyle": "-"
                                              },
                                 ax=ax)
    ax.grid(axis="y", zorder=-1, linewidth=0.4)

    ax.set_ylabel("")
    ax.set_xlabel("")
    ax.set_xticklabels(
        ax.get_xticklabels(),
        rotation=90,
    )

    ax.set_ylim([-0.025, 0.050])

    # ax.set_title(f"{tuning_x} VS {tuning_y}")

    sns.despine()
    plt.tight_layout(pad=0.5)

    plt.savefig(u.FIGURES_DIR / f"boxplot_tuningeffect_encoder_{tuning_x}_{tuning_y}.pdf")